# IMG01 : Get PostISRCCD

- author Sylvie Dagoret-Campagne
- creation date 2025-09-22 :
- affiliation : IJCLab
- Kernel @usdf **w_2025_38*
- Home emac : base (conda)
- laptop : conda_py313

DatasetType('postISRCCD', {band, instrument, day_obs, detector, group, physical_filter, exposure}, Exposure)

In [ ]:
dataType = 'postISRCCD'

**Goal** : Show Night variations of PWV wrt date and Time. Fit a straight line.

In [ ]:
# where are stored the figures
pathfigs = "figs_IMG01"
if not os.path.exists(pathfigs):
    os.makedirs(pathfigs) 
prefix = "img01"
figtype = ".png"

In [ ]:
FLAG_SAVEFITS = True
# where are stored the figures
pathdata = "data_IMG01"
if FLAG_SAVEFITS: 
    if not os.path.exists(pathdata):
        os.makedirs(pathdata) 
    imgtype = "fits"

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import lsst.afw.display as afwDisplay

In [ ]:

plt.rcParams["figure.figsize"] = (16,8)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'
plt.rcParams["legend.fontsize"] = "xx-large"

In [ ]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)


In [ ]:
# Remove to run faster the notebook
#import ipywidgets as widgets
#%matplotlib widget

In [ ]:
from IMG00_parameters import *

In [ ]:
DumpConfig()

## Butler

In [ ]:
import lsst.daf.butler as dafButler
from lsst.daf.butler import Butler

In [ ]:
butler = Butler(repo, collections=collection)

## Initialisation

### Read the file
- `atmfilename` is defined in `IMG00_parameters.py` 

In [ ]:
specdata = np.load(atmfilename,allow_pickle=True)

In [ ]:
df_spec = pd.DataFrame(specdata)

In [ ]:
# add time for plotting
df_spec["Time"] = pd.to_datetime(df_spec["DATE-OBS"])

In [ ]:
df_spec["seq_num"]  = df_spec["id"] % 100_000

In [ ]:
df_spec["nightObs"] = df_spec.apply(lambda x: x['id']//100_000, axis=1)

In [ ]:
df_spec[["id","FILTER"]]

In [ ]:
print(list(df_spec.columns))

## Suppress one Filter

In [ ]:
df_spec["FILTER"].unique() 

In [ ]:
df_spec = df_spec[df_spec.FILTER != "BG40_65mm_1"]

## night Selection

In [ ]:
df_spec["nightObs"].unique() 

In [ ]:
NIGHTSEL = 20250826

In [ ]:
df_sel = df_spec[df_spec["nightObs"] == NIGHTSEL]
df_sel = df_sel.reset_index(drop=True)

In [ ]:
df = df_sel[["id","seq_num","FILTER","PWV [mm]_x",'PWV [mm]_err_x','DATE-OBS','AIRMASS','CHI2_FIT']]
df["Time"] = pd.to_datetime(df["DATE-OBS"])

In [ ]:
df.head(n=20)

In [ ]:
YMIN,YMAX = 0.,15.
    
# Couleurs distinctes pour chaque filtre
colors = {
    filt: col for filt, col in zip(df["FILTER"].unique(), plt.cm.tab10.colors)
}

#plt.figure(figsize=(18,8))
fig,axs = plt.subplots(1,1,figsize=(18,8))
ax = axs

ax_bis = ax.twinx()  # crée un axe y secondaire partageant le même x
ax_bis.invert_yaxis()


# Plot par filtre avec barres d'erreur
for filt, group in df.groupby("FILTER"):
    ax.errorbar(
        group["Time"],
        group["PWV [mm]_x"],
        yerr=group["PWV [mm]_err_x"],
        fmt="o",
        label=filt,
        color=colors[filt],
        ecolor="k",
        capsize=5,
        markersize=10
    )
    
    # 🔎 Annoter chaque point avec son seq_num
    for x, y, seq in zip(group["Time"], group["PWV [mm]_x"], group["seq_num"]):
        ax.text(
            x, y + 0.15,  # léger décalage au-dessus du point
            str(seq),
            fontsize=12,fontweight="bold",
            ha="center"
        )
    ax_bis.scatter(group["Time"],group["AIRMASS"], color=colors[filt],marker="+",label="airmass")


ax_bis.set_ylim(2.5,0.5)
ax_bis.set_ylabel("airmass")

# Format de l'axe des X : jour + heure
#plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m-%d %H:%M"))
ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m-%d %H:%M"))

ax.set_xlabel("Time (UTC)")
ax.set_ylabel("PWV [mm]")
ax.set_title(f"PWV  for {NIGHTSEL} {tag}")
ax.legend(title="FILTER")
#ax.set_xticks(rotation=45)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")
ax.grid()
ax.set_ylim(YMIN,YMAX) 
plt.tight_layout()
plt.show()

## Extract the postISRCCD from the butler

In [ ]:
NMAX = 20

In [ ]:
len(df)

In [ ]:
df

In [ ]:
all_postisrccds = []
all_titles = []
all_md = []
ccd_dict = {}
for idx, row in df.iterrows():
    print(idx)
    if idx< NMAX:
        the_exposure = row["id"]
        print(idx,the_exposure)
        the_num = row["seq_num"]
        the_pwv = row["PWV [mm]_x"]
        the_pwv_err = row["PWV [mm]_err_x"]
        the_filter = row["FILTER"]
        the_title = f"{the_num} ({the_filter})"
        the_postisrccd = butler.get('postISRCCD', exposure=the_exposure, detector=0, collections=collection, instrument='LATISS')
        #raw_img= butler.get('raw', dataId={'exposure': the_exposure, 'instrument': 'LATISS', 'detector': 0}, collections = ['LATISS/calib','LATISS/raw/all',] )
        #meta = raw_img.getMetadata()
        meta = the_postisrccd.getMetadata()
        md = meta.toDict()
        all_titles.append(the_title)
        all_postisrccds.append(the_postisrccd )
        all_md.append(md)
        ccd_dict[the_num] = {"exposure": the_exposure, "filter":the_filter,"ccd": the_postisrccd,"md":md} 
    else:
        break

## Plots

In [ ]:
afwDisplay.setDefaultBackend("firefly")

In [ ]:
N = len(all_postisrccds)
for count in range(N):
    display = afwDisplay.Display(frame=count + 1)
    # cannot succeed to show white stars on dark sky
    display.setImageColormap("gray")
    display.scale("asinh", "zscale")
    display.mtv(all_postisrccds[count].image, title=all_titles[count])    

In [ ]:
# display.clearViewer()

## Save fits

In [ ]:
list_of_seqnum = [189,197,205]

In [ ]:
if FLAG_SAVEFITS:
    for num in list_of_seqnum:
        ccdimg = ccd_dict[num]["ccd"]
        expos = ccd_dict[num]["exposure"]
        filt = ccd_dict[num]["filter"]
        filename = f"{prefix}_postisrccd_{filt}_{expos}.{imgtype}"
        fullfilename = os.path.join(pathdata,filename)
        print(f"Save image file {fullfilename}")
        ccdimg.writeFits(fullfilename) 